In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 11,padding="same") #input -28*28 OUtput = 32 *28*28   RF = 3*3
        self.pool1 = nn.MaxPool2d(2, 2)  
        self.conv2 = nn.Conv2d(32, 64, 1, padding="same") # make padding to zero because we don't know whether to keep one or not  && RF = 5*5
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 1, padding="same")
        self.avgpool=  nn.AvgPool2d(7)
        self.fc    = nn.Linear(128,10)

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x =  self.dropout(self.pool1(F.relu(self.conv1(x))))
        x = self.dropout(self.pool2(F.relu(self.conv2(x))))
        x = self.avgpool(F.relu(self.conv3(x)))
        # x = F.relu(self.conv7(x)) # should not apply relu before output so removed it
        x = x.view(-1, 128)
        x = self.fc(x)

        return F.log_softmax(x, dim = 1)

In [15]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") 
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]           3,904
         MaxPool2d-2           [-1, 32, 14, 14]               0
           Dropout-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]           2,112
         MaxPool2d-5             [-1, 64, 7, 7]               0
           Dropout-6             [-1, 64, 7, 7]               0
            Conv2d-7            [-1, 128, 7, 7]           8,320
         AvgPool2d-8            [-1, 128, 1, 1]               0
            Linear-9                   [-1, 10]           1,290
Total params: 15,626
Trainable params: 15,626
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.48
P

In [18]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 6, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [23]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [27]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.95)

for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.3058968484401703 batch_id=234: 100%|██████████| 235/235 [00:11<00:00, 20.52it/s]



Test set: Average loss: 0.2440, Accuracy: 9257/10000 (93%)



loss=0.20285789668560028 batch_id=234: 100%|██████████| 235/235 [00:11<00:00, 20.47it/s]



Test set: Average loss: 0.1400, Accuracy: 9581/10000 (96%)



loss=0.16299481689929962 batch_id=234: 100%|██████████| 235/235 [00:11<00:00, 20.42it/s]


...


Test set: Average loss: 0.0426, Accuracy: 9867/10000 (99%)



loss=0.025841303169727325 batch_id=234: 100%|██████████| 235/235 [00:11<00:00, 19.84it/s]



Test set: Average loss: 0.0489, Accuracy: 9852/10000 (99%)



loss=0.03420846909284592 batch_id=234: 100%|██████████| 235/235 [00:11<00:00, 19.85it/s]



Test set: Average loss: 0.0396, Accuracy: 9875/10000 (99%)

